In [10]:
import SQcircuit_extensions as sq_ext
import numpy as np
import matplotlib.pyplot as plt

%matplotlib ipympl

## KIT's qubit

This notebook studies the qubit proposed by KIT. It's lumped element model is:
<img src='Single qubit 4 nodes CC coupled to ground.jpg' width='500'>


## Parameters

In [11]:
n_max_resonator = 50
n_max_fluxonium = 50

trunc_nums_resonator = [n_max_resonator]
trunc_nums_fluxonium = [n_max_fluxonium]

trunc_nums = [1, n_max_resonator, n_max_fluxonium]
trunc_nums_Δ_0 = [ n_max_resonator, n_max_fluxonium]
φ_ext = 0.5
Δ_values = np.array([1e-10, 1e-3, 1e-1])
n_eig = 2

## Circuit vs Δ study

In [12]:
qubit_vs_Δ =     [ ]
resonator_vs_Δ = [ ]
fluxonium_vs_Δ = [ ]
H_frc_vs_Δ = [ ]

for Δ in Δ_values:

    qubit     = sq_ext.KIT_qubit_no_JJ    (Δ=Δ)
    fluxonium = sq_ext.KIT_fluxonium_no_JJ(Δ=Δ)
    # qubit     = sq_ext.sq_qubit    (Δ=Δ)
    # fluxonium = sq_ext.sq_fluxonium(Δ=Δ)
    resonator = sq_ext.sq_resonator(Δ=Δ)

    resonator.set_trunc_nums(trunc_nums_resonator)
    fluxonium.set_trunc_nums(trunc_nums_fluxonium)

    try:
        qubit.set_trunc_nums(trunc_nums)
    except:
        try:
            qubit.set_trunc_nums(trunc_nums_Δ_0)
        except:
            qubit.set_trunc_nums([trunc_nums_resonator[0]*trunc_nums_fluxonium[0]])

    #This try and except structure is because if you use the "no_JJ" circuits there is no loop.
    try:
        fluxonium.loops[0].set_flux(φ_ext)
        qubit.loops[0].set_flux(φ_ext)
    except:
        pass

    _, _ = resonator.diag(n_eig)
    _, _ = fluxonium.diag(n_eig)
    _, _ = qubit.diag(n_eig)

    qubit_vs_Δ    .append(qubit)
    fluxonium_vs_Δ.append(fluxonium)
    resonator_vs_Δ.append(resonator)
    H_frc_vs_Δ    .append(sq_ext.hamiltonian_qubit(fluxonium, resonator, Δ))

In [13]:
qubit_vs_Δ[0].description(), sq_ext.print_flux_transformation(qubit_vs_Δ[0])

In [15]:
H_eff_full = sq_ext.H_eff_p1(qubit_vs_Δ[0], qubit_vs_Δ[-1], real='True')
H_eff_full -= np.eye(len(H_eff_full)) * H_eff_full[0,0]
H_eff_full

In [14]:
qubit_vs_Δ[-1].description(), sq_ext.print_flux_transformation(qubit_vs_Δ[-1])

In [16]:
H_eff_frc = sq_ext.H_eff_p1_hamil(H_frc_vs_Δ[0],H_frc_vs_Δ[-1],n_eig , real='True')
H_eff_frc -= np.eye(len(H_eff_frc)) * H_eff_frc[0,0]
H_eff_frc